In [1]:
import pandas as pd
import folium
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from folium.plugins import HeatMap
from global_land_mask import globe
from collections import defaultdict
import geopandas as gpd
import pygeohash as pgh
import seaborn as sns
from shapely.geometry import Polygon
import branca.colormap
from matplotlib import rcParams
from functools import reduce

In [2]:
x = pd.read_csv("2018_Conainer_visData.csv")[["imo","hour","long_x","lat_x","Size_Bin"]]
x

,imo,hour,long_x,lat_x,Size_Bin
0,7033161,0,-99999.000000,-99999.000000,1
1,7033161,1,-99999.000000,-99999.000000,1
2,7033161,2,-99999.000000,-99999.000000,1
3,7033161,3,-99999.000000,-99999.000000,1
4,7033161,4,-99999.000000,-99999.000000,1
...,...,...,...,...,...
45131650,9854662,8755,125.173790,1.425128,1
45131651,9854662,8756,125.173698,1.425256,1
45131652,9854662,8757,125.173840,1.425092,1
45131653,9854662,8758,125.173882,1.424985,1


In [3]:
x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]
x

<ipython-input-3-ea7ed866ecf4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]
<ipython-input-3-ea7ed866ecf4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]


,imo,hour,long_x,lat_x,Size_Bin
35040,7617890,0,-139.536276,27.388975,3
35041,7617890,1,-139.043204,27.545672,3
35042,7617890,2,-138.731787,27.644878,3
35043,7617890,3,-138.562793,27.712152,3
35044,7617890,4,-138.259306,27.914180,3
...,...,...,...,...,...
45097355,9843194,740,120.909735,31.852588,1
45097356,9843194,741,120.909763,31.852387,1
45097357,9843194,742,120.909762,31.852267,1
45097358,9843194,743,120.909717,31.852131,1


In [4]:
x = x[["imo","long_x","lat_x","Size_Bin"]]
x = x.sample(100000)
x

,imo,long_x,lat_x,Size_Bin
41715860,9728007,108.299536,12.504683,2
33428544,9478494,122.501535,32.124041,4
26464722,9384875,100.607916,13.318700,2
29434345,9420368,52.839525,26.870220,2
41925506,9732333,7.292195,54.154307,8
...,...,...,...,...
15838543,9295256,63.504547,11.568714,6
39026394,9665619,104.702063,2.035715,7
41671901,9726671,37.586896,23.014339,6
35890246,9558488,116.029475,-5.518919,1


In [5]:
#plot markers for the top 10 bunkering ports on the map
def addBunker(vis):
    folium.Marker(location=[1.3521,103.8198],popup='Singapore',tooltip='Singapore',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[51.949,4.153],popup='Rotterdam',tooltip='Rotterdam',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[25.175,56.3586],popup='Fujairah',tooltip='Fujairah',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[22.2932,114.1613],popup='Hong Knog',tooltip='Hong Kong',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[51.2411,4.403],popup='Antwerp',tooltip='Antwerp',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[35.103,129.0424],popup='Busan',tooltip='Busan',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[36.133,-5.35],popup='Gibraltar',tooltip='Gibraltar',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[8.9575,-79.561],popup='Panama',tooltip='Panama',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[36.1252,-5.4395],popup='Algeciras',tooltip='Algeciras',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[33.740,-118.281],popup='Los Angeles',tooltip='Los Angeles',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    
    return vis
#For each lat,long pair in the dataset, calculaate geohash values and return the centroid of the geohash blocks
# More info on geohash: https://www.movable-type.co.uk/scripts/geohash.html
def getCentroid(x):
    l=x.split(',')
    lat=float(l[0])
    lon=float(l[1])
    prec=4
    gh=pgh.encode(lat,lon,precision=prec)
    return pgh.decode(gh)
sbins=[1,2,3,4,5,6,7,8,9]
#Check if a lat, long pair is on land, returns True/False
def isinLand(x):
    #l=x.split(',')
    lat=float(x[0])
    lon=float(x[1])
    return globe.is_land(lat,lon)

In [6]:

min= x[["Size_Bin"]].min()
max = x[["Size_Bin"]].max()
print(min,max)

Size_Bin    1
dtype: int64 Size_Bin    9
dtype: int64


In [7]:
import branca.colormap as cm
blRd = cm.LinearColormap(['red', 'green'],
                           vmin=1, vmax=9)

blRd

In [8]:
#create a map
this_map = folium.Map(prefer_canvas=True)

#create a color map
color_var = 'Size_Bin' #what variable will determine the color
cmap = cm.LinearColormap(['red', 'green'],
                         vmin=x[color_var].quantile(0.05), vmax=x[color_var].quantile(0.95),
                         caption = color_var)

#Add the color map legend to your map
this_map.add_child(cmap)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.lat_x, point.long_x],
                        fill_color=cmap(point[color_var]),
                        radius=2,
                        weight=0).add_to(this_map)

#use df.apply(,axis=1) to iterate through every row in your dataframe
x.apply(plotDot, axis = 1)

#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file


this_map
import os

this_map.save(os.path.join("/Users/zwt2000/Desktop/Ship_Visualisation/","Locationbysize.html"))

